In [1]:
from pyspark.sql import SparkSession
from utils.pysparkutils import *

spark = SparkSession.builder.appName('test').getOrCreate()

In [2]:
ddd = {'a': 1, 'b': 2}
d = FrozenDict(ddd)
list(d.items())

[('a', 1), ('b', 2)]

In [3]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import format_number

def crosstabPercentage(df, col1, col2):
    ctabDf = df.crosstab(col1, col2)
    ctabCol = col1 + '_' + col2
    ctabNewCol = col1 + col2.title()
    ctabDf = ctabDf.withColumn(ctabNewCol, ctabDf[ctabCol])\
                                         .orderBy(ctabNewCol).drop(ctabCol)
    # Strip extra whitespaces from column name
    for column in ctabDf.columns:
        columnStripped = column.strip()
        if column != columnStripped:
            ctabDf = ctabDf.withColumn(column.strip(), ctabDf[column])\
                                         .drop(column)
    cols = ctabDf.columns[:]
    cols.remove(ctabNewCol)
    ctabDf = ctabDf.select(ctabNewCol, *cols)
    ctabDf.show()

In [4]:
%%script false

from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

from pyspark.sql.types import *
import pandas as pd

df = spark.createDataFrame([
    ("a", 0), ("a", 1), ("b", 30), ("b", -50)
], ["group", "power"])

def below_threshold(group="group", power="power"):
    @pandas_udf("struct<group: string, sum: int>", PandasUDFType.GROUPED_MAP)
    def below_threshold_(df):
        df = pd.DataFrame(
           df.groupby(group).apply(df[power].sum()))
        df.reset_index(inplace=True, drop=False)
        return df

    return below_threshold_

df.groupBy("group").apply(below_threshold()).show()

In [5]:
from pyspark.sql.functions import col, countDistinct, lit
from pyspark.ml.linalg import Vectors
import math
import scipy.stats
from utils.pysparkutils import *

df = spark.createDataFrame([
    ("a", 0), 
    ("a", 1), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", 30), 
    ("b", -50),
], ["group", "power"])

# calcIndividualAndJointPorbablities(df, 'group', 'power')
calcMutualInformation(df, 'group', 'power')

0.5297061990576546

In [6]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import count, rank, min

# windowSpec =  Window.orderBy('id2', 'id3')
# windowSpec = windowSpec.rowsBetween(Window.unboundedPreceding, Window.currentRow)
# # df.select(count('id3').over(w).alias('count')).show()
# df.select(min(labelCol).over(w)).show()

# df.crosstab("id1", "id2").show()
# crosstabPercentage(df, "id1", "id2")

# df2 = df.groupby(labelCol, "id2").count()
# count = df.count()
# df2.withColumn('percentage', col('count') / count).show()

In [7]:
grouped = df.groupby("id")
grouped

In [8]:
grouped.

SyntaxError: invalid syntax (<ipython-input-8-0eb7437abb25>, line 1)